# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments          Customer  \
0  Jan  6 2023  9:58AM  254447        19      8997614    Graystone, LLC   
1  Jan  6 2023  9:56AM  254446        19  PBIO-103655   Paragon BioTeck   
2  Jan  6 2023  9:56AM  254446        19  PBIO-103756   Paragon BioTeck   
3  Jan  6 2023  9:54AM  254445        10   CTF0011574    Citieffe, Inc.   
4  Jan  6 2023  9:48AM  254444        10   8989013-BO  Eywa Pharma Inc.   
5  Jan  6 2023  9:48AM  254444        10      8994548  Eywa Pharma Inc.   
6  Jan  6 2023  9:48AM  254444        10      8994551  Eywa Pharma Inc.   
7  Jan  6 2023  9:48AM  254444        10      8994552  Eywa Pharma Inc.   
8  Jan  6 2023  9:48AM  254444        10      8997096  Eywa Pharma Inc.   
9  Jan  6 2023  9:48AM  254444        10      8994528  Eywa Pharma Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
52  254440       Released          1
53  254444       Released          6
54  254445       Released          1
55  254446       Released          2
56  254447       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
254440                NaN        NaN       1.0
254444                NaN        NaN       6.0
254445                NaN        NaN       1.0
254446                NaN        NaN       2.0
254447                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
254331                0.0        0.0       1.0
254346                0.0        0.0       1.0
254347                0.0        0.0       1.0
254348                0.0        1.0       0.0
254351                0.0        0.0       2.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
254331                  0          0         1
254346                  0          0         1
254347                  0          0         1
254348                  0          1         0
254351                  0          0         2

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               254331          0          0         1
1               254346          0          0         1
2               254347          0          0         1
3               254348          0          1         0
4               254351          0          0         2

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               254331                            1
1               254346                            1
2               254347                            1
3               254348                   1         
4               254351                            2

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse           Customer
0   Jan  6 2023  9:58AM  254447        19     Graystone, LLC
1   Jan  6 2023  9:56AM  254446        19    Paragon BioTeck
3   Jan  6 2023  9:54AM  254445        10     Citieffe, Inc.
4   Jan  6 2023  9:48AM  254444        10   Eywa Pharma Inc.
10  Jan  6 2023  9:12AM  254439        10     Citieffe, Inc.
12  Jan  6 2023  9:05AM  254440        10            Bio-PRF
13  Jan  6 2023  8:54AM  254436        10  ISDIN Corporation
73  Jan  6 2023  8:50AM  254438        10        Emerginnova
75  Jan  6 2023  8:50AM  254437        10  ISDIN Corporation
98  Jan  6 2023  8:44AM  254432        10  ISDIN Corporation

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse           Customer Completed  \
0  Jan  6 2023  9:58AM  254447        19     Graystone, LLC             
1  Jan  6 2023  9:56AM  254446        19    Paragon BioTeck             
2  Jan  6 2023  9:54AM  254445        10     Citieffe, Inc.             
3  Jan  6 2023  9:48AM  254444        10   Eywa Pharma Inc.             
4  Jan  6 2023  9:12AM  254439        10     Citieffe, Inc.             
5  Jan  6 2023  9:05AM  254440        10            Bio-PRF             
6  Jan  6 2023  8:54AM  254436        10  ISDIN Corporation             
7  Jan  6 2023  8:50AM  254438        10        Emerginnova             
8  Jan  6 2023  8:50AM  254437        10  ISDIN Corporation             
9  Jan  6 2023  8:44AM  254432        10  ISDIN Corporation             

  Executing Released  
0                  1  
1                  2  
2                  1  
3                  6  
4                  2  
5                  1  
6                 60  
7                  2  
8                 23  
9                  9

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse           Customer Released  \
0  Jan  6 2023  9:58AM  254447        19     Graystone, LLC        1   
1  Jan  6 2023  9:56AM  254446        19    Paragon BioTeck        2   
2  Jan  6 2023  9:54AM  254445        10     Citieffe, Inc.        1   
3  Jan  6 2023  9:48AM  254444        10   Eywa Pharma Inc.        6   
4  Jan  6 2023  9:12AM  254439        10     Citieffe, Inc.        2   
5  Jan  6 2023  9:05AM  254440        10            Bio-PRF        1   
6  Jan  6 2023  8:54AM  254436        10  ISDIN Corporation       60   
7  Jan  6 2023  8:50AM  254438        10        Emerginnova        2   
8  Jan  6 2023  8:50AM  254437        10  ISDIN Corporation       23   
9  Jan  6 2023  8:44AM  254432        10  ISDIN Corporation        9   

  Executing Completed  
0                      
1                      
2                      
3                      
4                      
5                      
6                      
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse          Customer Released Executing  \
0  Jan  6 2023  9:58AM  254447        19    Graystone, LLC        1             
1  Jan  6 2023  9:56AM  254446        19   Paragon BioTeck        2             
2  Jan  6 2023  9:54AM  254445        10    Citieffe, Inc.        1             
3  Jan  6 2023  9:48AM  254444        10  Eywa Pharma Inc.        6             
4  Jan  6 2023  9:12AM  254439        10    Citieffe, Inc.        2             

  Completed  
0            
1            
2            
3            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse          Customer  Released  \
0  Jan  6 2023  9:58AM  254447        19    Graystone, LLC       1.0   
1  Jan  6 2023  9:56AM  254446        19   Paragon BioTeck       2.0   
2  Jan  6 2023  9:54AM  254445        10    Citieffe, Inc.       1.0   
3  Jan  6 2023  9:48AM  254444        10  Eywa Pharma Inc.       6.0   
4  Jan  6 2023  9:12AM  254439        10    Citieffe, Inc.       2.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse          Customer  Released  \
0  Jan  6 2023  9:58AM  254447        19    Graystone, LLC       1.0   
1  Jan  6 2023  9:56AM  254446        19   Paragon BioTeck       2.0   
2  Jan  6 2023  9:54AM  254445        10    Citieffe, Inc.       1.0   
3  Jan  6 2023  9:48AM  254444        10  Eywa Pharma Inc.       6.0   
4  Jan  6 2023  9:12AM  254439        10    Citieffe, Inc.       2.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         5596969     346.0       12.0       11.0
15          508757       5.0        0.0        0.0
16         1272050       3.0        2.0        0.0
18          254420       1.0        0.0        0.0
19         3307341      26.0        2.0        4.0
20          763182       3.0        0.0        0.0
21         1780620       5.0        2.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  5596969     346.0       12.0       11.0
1        15   508757       5.0        0.0        0.0
2        16  1272050       3.0        2.0        0.0
3        18   254420       1.0        0.0        0.0
4        19  3307341      26.0        2.0        4.0
5        20   763182       3.0        0.0        0.0
6        21  1780620       5.0        2.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     346.0       12.0       11.0
1        15       5.0        0.0        0.0
2        16       3.0        2.0        0.0
3        18       1.0        0.0        0.0
4        19      26.0        2.0        4.0
5        20       3.0        0.0        0.0
6        21       5.0        2.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  346.0
1        15  Released    5.0
2        16  Released    3.0
3        18  Released    1.0
4        19  Released   26.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10   15   16   18    19   20   21
Status                                         
Completed   11.0  0.0  0.0  0.0   4.0  0.0  0.0
Executing   12.0  0.0  2.0  0.0   2.0  0.0  2.0
Released   346.0  5.0  3.0  1.0  26.0  3.0  5.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10   15   16   18    19   20   21
0          Completed   11.0  0.0  0.0  0.0   4.0  0.0  0.0
1          Executing   12.0  0.0  2.0  0.0   2.0  0.0  2.0
2           Released  346.0  5.0  3.0  1.0  26.0  3.0  5.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10   15   16   18    19   20   21
0  Completed   11.0  0.0  0.0  0.0   4.0  0.0  0.0
1  Executing   12.0  0.0  2.0  0.0   2.0  0.0  2.0
2   Released  346.0  5.0  3.0  1.0  26.0  3.0  5.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()